# Resumen: Web Scraping de Reseñas desde IMDb con **Selenium** y **BeautifulSoup**

## Pasos
1. **Inicializar**:
   - Crear listas vacías (`reseñas_totales` y `peliculas_sin_reseñas`) para almacenar datos.
   - Configurar un contador para las películas sin reseñas.

2. **Automatización con Selenium**:
   - Iterar sobre los títulos de películas y buscar cada uno en el campo de búsqueda de IMDb.
   - Seleccionar el primer título sugerido y realizar la búsqueda.

3. **Extraer reseñas**:
   - Intentar localizar el botón de reseñas y hacer clic en él.
   - Obtener los artículos de reseñas disponibles y extraer su título y comentario.
   - Guardar las reseñas extraídas en `reseñas_totales`.

4. **Manejo de errores**:
   - Si no hay reseñas disponibles, registrar la película en `peliculas_sin_reseñas` con un mensaje indicando la ausencia de datos.
   - Manejar excepciones para errores de búsqueda o extracción.

5. **Convertir y guardar resultados**:
   - Crear un DataFrame de pandas con los datos recopilados.
   - Guardar el DataFrame en un archivo CSV.

#### Librerías

In [31]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

#### Títulos de las películas a scrapear

In [32]:
titles_data = pd.read_csv("C:\\Users\\Usuario\\Desktop\\Netflix\\Raw_data\\titulos_pendientes_scrapear_12.csv")
titles_data.rename(columns={'0': 'title'}, inplace=True)
titles_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  383 non-null    int64 
 1   title       383 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.1+ KB


In [33]:
titles_data

,Unnamed: 0,title
0,300,Titipo Titipo
1,301,"Out of Many, One"
2,302,Soa
3,303,Wentworth
4,304,Dreamworks How to Train Your Dragon Legends
...,...,...
378,678,Side Dish
379,679,Agustín Aristarán: Soy Rada
380,680,Jani Dueñas: Grandes fracasos de ayer y hoy
381,681,The Hook Up Plan


In [34]:
# bucle para obtener los titulos de las películas a scrapear
titulos = []
for titles in titles_data["title"]:
    titulos.append(titles)

# lista de titulos a scrapear
titulos

['Titipo Titipo',
 'Out of Many, One',
 'Soa',
 'Wentworth',
 'Dreamworks How to Train Your Dragon Legends',
 'Bonus Family',
 'Dorohedoro',
 'Secret City',
 'Mom',
 'The Umbrella Academy',
 'Manorama Six Feet Under',
 'Sitara',
 'Paskal',
 'Control Z',
 'Leila',
 "Komi Can't Communicate",
 'Cuckoo',
 'The Beginning of Life',
 'The Hows of Us',
 'Spyder',
 'Danger Force',
 'The Light in Your Eyes',
 'Silence Is Welcome',
 'Umrika',
 '뽀로로 동화나라',
 'One Take',
 'Bashar Shorts',
 'Street Flow',
 'Tea Time',
 'Afronta!',
 'Power Rangers Ninja Steel',
 "Monty Python's Fliegender Zirkus",
 'Two Fathers',
 'Fastest Car',
 'Monty Python Conquers America',
 'Another Self',
 'Lou',
 'Chloe',
 'Patriot Act with Hasan Minhaj',
 'I Am a Killer',
 'Brotherhood',
 'Audrie & Daisy',
 'Hunterrr',
 'Move',
 'The Stranger',
 'Team Kaylie',
 "Summoning Amr's Guardian",
 'Prince of Peoria',
 'Inventing David Geffen',
 'Alba',
 'Fakes',
 'Party Monster: Scratching the Surface',
 'Lakshya',
 'Roxanne Roxanne'

### Rutas y navegador

In [35]:
# Ruta del GeckoDriver
driver_path = 'C:/Users/Usuario/Desktop/Master Data& IA/APIS y Web scraping/chromedriver firefoxdriver/geckodriver.exe'

# Ruta del ejecutable de Firefox
firefox_binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'

# Configurar las opciones de Firefox
firefox_options = Options()
firefox_options.binary_location = firefox_binary_path

# Inicializar el driver
service = Service(driver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)

# Cargar una página de prueba
driver.get("https://www.imdb.com/es-es/?ref_=nv_home")

# Cerrar el driver después de unos segundos
import time
time.sleep(1)

###  Aceptar Cookies

In [36]:
# Esperar que el botón de aceptar cookies esté visible y clicarlo
try:
    aceptar_cookies = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.icb-btn:nth-child(2)"))
    )
    aceptar_cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("El botón de cookies no se encontró o ya estaba aceptado.")

Cookies aceptadas.


### Lógica del scraping

In [37]:
# Inicialización de listas
reseñas_totales = []
peliculas_sin_reseñas = []
contador = 0


def extraer_comentarios(driver, titulo_pelicula):
    """Extrae los comentarios de las reseñas de una película."""
    try:
        articulos_resenas = driver.find_elements(By.CSS_SELECTOR, "article.sc-7d2e5b85-1")
        if not articulos_resenas:
            print(f"No se encontraron reseñas para '{titulo_pelicula}'.")
            peliculas_sin_reseñas.append(titulo_pelicula)
            return

        for articulo in articulos_resenas:
            try:
                # Extraer título de la reseña
                titulo_resena = "Título no disponible"
                try:
                    titulo_resena = articulo.find_element(By.CSS_SELECTOR, "h3").text.strip()
                except:
                    pass

                # Extraer comentario de la reseña
                comentario_resena = "Comentario no disponible"
                try:
                    comentario_elemento = articulo.find_element(By.CSS_SELECTOR, "div.ipc-html-content-inner-div")
                    comentario_resena = comentario_elemento.text.strip()
                except:
                    pass

                # Almacenar los datos
                reseñas_totales.append({
                    "title": titulo_pelicula,
                    "title_comment": titulo_resena,
                    "comment": comentario_resena
                })
            except Exception as e:
                print(f"Error al procesar una reseña: {e}")
    except Exception as e:
        print(f"Error al extraer comentarios: {e}")

for titulo_pelicula in titulos:
    print(f"Buscando el título: {titulo_pelicula}")
    
    try:
        # Esperar a que el campo de búsqueda esté presente
        campo_busqueda = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#suggestion-search"))
        )
        
        # Limpiar el campo y escribir el título
        campo_busqueda.clear()
        campo_busqueda.send_keys(titulo_pelicula)
        print(f"Título '{titulo_pelicula}' ingresado en el campo de búsqueda.")
        
        time.sleep(1.5)  # Pausa breve para permitir la selección
        campo_busqueda.send_keys(Keys.ARROW_DOWN)  # Seleccionar primer título sugerido
        time.sleep(1.5)  # Pausa breve
        campo_busqueda.send_keys(Keys.RETURN)  # Presionar "Enter"
        print("Primer título seleccionado y búsqueda ejecutada.")
        time.sleep(2)  # Esperar a que cargue la página
        
        # Intentar encontrar el botón de reseñas
        try:
            boton_resenas = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".sc-1fdae676-0 > li:nth-child(2) > a:nth-child(1)"))
            )
            boton_resenas.click()
            print("Botón de reseñas clicado correctamente.")
            
            time.sleep(2)  # Espera para que cargue la sección de reseñas
            
            # Extraer las reseñas de la película actual
            extraer_comentarios(driver, titulo_pelicula)
        
        except Exception:
            print(f"La película '{titulo_pelicula}' NO TIENE RESEÑAS.")
            reseñas_totales.append({
                "title": titulo_pelicula,
                "Título_commentario": "No disponible",
                "Comentario": "Esta película no tiene reseñas disponibles."
            })
            peliculas_sin_reseñas.append(titulo_pelicula)  
            contador += 1
    except Exception as e:
        print(f"¡¡¡¡¡ERROR AL PROCESAR UNA PELÍCULA!!!!!!!! '{titulo_pelicula}': {e}")

# Convertir los datos totales a un DataFrame
df_reseñas_totales = pd.DataFrame(reseñas_totales)

Buscando el título: Titipo Titipo
Título 'Titipo Titipo' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
Botón de reseñas clicado correctamente.
No se encontraron reseñas para 'Titipo Titipo'.
Buscando el título: Out of Many, One
Título 'Out of Many, One' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película 'Out of Many, One' NO TIENE RESEÑAS.
Buscando el título: Soa
Título 'Soa' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película 'Soa' NO TIENE RESEÑAS.
Buscando el título: Wentworth
Título 'Wentworth' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
Botón de reseñas clicado correctamente.
No se encontraron reseñas para 'Wentworth'.
Buscando el título: Dreamworks How to Train Your Dragon Legends
Título 'Dreamworks How to Train Your Dragon Legends' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.


In [38]:
# Resumen final
print(f"\nProceso completado. Total de películas procesadas: {titulos}")
print(f"Total de películas sin reseñas: {contador}")
print("Películas sin reseñas:")
print(peliculas_sin_reseñas)


Proceso completado. Total de películas procesadas: ['Titipo Titipo', 'Out of Many, One', 'Soa', 'Wentworth', 'Dreamworks How to Train Your Dragon Legends', 'Bonus Family', 'Dorohedoro', 'Secret City', 'Mom', 'The Umbrella Academy', 'Manorama Six Feet Under', 'Sitara', 'Paskal', 'Control Z', 'Leila', "Komi Can't Communicate", 'Cuckoo', 'The Beginning of Life', 'The Hows of Us', 'Spyder', 'Danger Force', 'The Light in Your Eyes', 'Silence Is Welcome', 'Umrika', '뽀로로 동화나라', 'One Take', 'Bashar Shorts', 'Street Flow', 'Tea Time', 'Afronta!', 'Power Rangers Ninja Steel', "Monty Python's Fliegender Zirkus", 'Two Fathers', 'Fastest Car', 'Monty Python Conquers America', 'Another Self', 'Lou', 'Chloe', 'Patriot Act with Hasan Minhaj', 'I Am a Killer', 'Brotherhood', 'Audrie & Daisy', 'Hunterrr', 'Move', 'The Stranger', 'Team Kaylie', "Summoning Amr's Guardian", 'Prince of Peoria', 'Inventing David Geffen', 'Alba', 'Fakes', 'Party Monster: Scratching the Surface', 'Lakshya', 'Roxanne Roxanne',

In [39]:
df_reseñas_totales = df_reseñas_totales[["title", "title_comment", "comment"]]

In [40]:
df_reseñas_totales["comment"].duplicated().sum()


435

In [41]:
df_reseñas_totales

,title,title_comment,comment
0,"Out of Many, One",NaN,NaN
1,Soa,NaN,NaN
2,Dreamworks How to Train Your Dragon Legends,NaN,NaN
3,Bonus Family,Another NFLX Homerun,An absolutely entertaining uncomfortable wonde...
4,Bonus Family,Big Irish Fan,I'm a big fan of anything scandanavian . Great...
...,...,...,...
1275,On My Block,Finally a show acuratley representing youth cu...,From the way the kids speak to the background ...
1276,On My Block,Great show,The dialogue is natural and full of quick witt...
1277,On My Block,"Good TV Show with talented, young actors",This show has great potential! The scoring is ...
1278,On My Block,Total disappointment,The show started out great living on planet ea...


### Exportación CSV

In [42]:
# Guardar en CSV
df_reseñas_totales.to_csv("scrap_ultimo_15.csv", index=False, encoding="utf-8")